In [1]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
from glob import glob
from random import randint, sample
from tqdm import tqdm

In [2]:
data8 = pd.read_csv('./new_label8.csv', header=None, names=['id1', 'id2', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
data9 = pd.read_csv('./new_label9.csv', header=None, names=['id1', 'id2', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])
data10 = pd.read_csv('./new_label10.csv', header=None, names=['id1', 'id2', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'])

data = pd.concat([data8, data9, data10], ignore_index=True)
data

,id1,id2,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00018,TRAIN_21645,1,1,1,1,0,1,0,1,1,1
1,TRAIN_00016,TRAIN_21429,1,1,1,1,1,1,0,1,0,1
2,TRAIN_00029,TRAIN_20685,1,1,1,1,1,1,1,0,1,0
3,TRAIN_00045,TRAIN_20853,1,1,1,1,1,0,1,0,1,1
4,TRAIN_00071,TRAIN_21861,0,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9810,TRAIN_22177,TRAIN_22199,1,1,1,1,1,1,1,1,1,1
9811,TRAIN_22194,TRAIN_22200,1,1,1,1,1,1,1,1,1,1
9812,TRAIN_22198,TRAIN_22200,1,1,1,1,1,1,1,1,1,1
9813,TRAIN_22198,TRAIN_22199,1,1,1,1,1,1,1,1,1,1


In [3]:
bg_lst = []
bg_paths = glob('../data/crawed_img/*/*.jpg')
with open('./new_used_name.txt', 'r') as f:
    lines = f.readlines()
    used_name_lst = [line.strip() for line in lines]

for path in bg_paths:
    if path not in used_name_lst:
        bg_lst.append(path)

In [4]:
print(len(bg_paths)==len(bg_lst)+len(used_name_lst))

True


In [5]:
len(bg_lst)

13829

In [6]:
for idx, row in tqdm(data.iterrows()):
    img_path1 = os.path.join('../data/train', f"{row.id1}.jpg")
    img1 = cv2.imread(img_path1)
    # img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    
    img_path2 = os.path.join('../data/train', f"{row.id2}.jpg")
    img2 = cv2.imread(img_path2)
    # img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    thresh = 230
    
    bg = cv2.imread(bg_lst[idx])
    # bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    
    
    if np.random.uniform() > 0.8: # 1
        random_h = randint(70, 80) 
        random_w = randint(70, 80)
    elif np.random.uniform() > 0.6: # 2
        random_h = randint(70, 80) 
        random_w = randint(-80, -70)
    elif np.random.uniform() > 0.4: # 3
        random_h = randint(-80, -70) 
        random_w = randint(70, 80)
    else: # 4
        random_h = randint(-80, -70) 
        random_w = randint(-80, -70)
    
    
    h, w, c = img1.shape
    #####1
    resize_img1= cv2.resize(cv2.copyMakeBorder(img1,50,50,50,50,cv2.BORDER_CONSTANT,value=(255, 255, 255)), (h, w))
    move_img1 = imutils.translate(resize_img1, random_w, random_h)
    
    move_img1[np.where(move_img1 == 0)] = 255
    img_bin1 = np.array(move_img1[:, :, 0] < thresh, dtype=np.uint8) * 255
    img_blur1 = cv2.medianBlur(img_bin1, 5)
    img_filter1 = move_img1.copy()
    img_filter1[img_blur1 == 0] = 0
    
    retval, labels1, stats1, centroids1 = cv2.connectedComponentsWithStats(img_blur1)
    stats1 = np.array(stats1)
    stats1[0, -1] = 0
    label_idx1 = np.argmax(stats1[:, -1])
    img_mask1 = labels1 == label_idx1
    img_final1 = img_filter1.copy()
    img_final1[~img_mask1] = (0, 0, 0)
    
    #####2
    resize_img2= cv2.resize(cv2.copyMakeBorder(img2,50,50,50,50,cv2.BORDER_CONSTANT,value=(255, 255, 255)), (h, w))
    move_img2 = imutils.translate(resize_img2, -1*random_w, -1*random_h)
    
    move_img2[np.where(move_img2 == 0)] = 255
    img_bin2 = np.array(move_img2[:, :, 0] < thresh, dtype=np.uint8) * 255
    img_blur2 = cv2.medianBlur(img_bin2, 5)
    img_filter2 = move_img2.copy()
    img_filter2[img_blur2 == 0] = 0
    
    retval, labels2, stats2, centroids2 = cv2.connectedComponentsWithStats(img_blur2)
    stats2 = np.array(stats2)
    stats2[0, -1] = 0
    label_idx2 = np.argmax(stats2[:, -1])
    img_mask2 = labels2 == label_idx2
    img_final2 = img_filter2.copy()
    img_final2[~img_mask2] = (0, 0, 0)

    ###
    img_back = cv2.resize(bg, (h, w))
    img_back[img_mask1] = (0, 0, 0)
    img_back[img_mask2] = (0, 0, 0)
    
    img_final1 += img_final2
    img_final1 += img_back

    img_name = f'TRAIN_{32994+idx}'
    cv2.imwrite(f'../data/new_merge_train/{img_name}.jpg', img_final1)
    with open('../data/new_train.csv', 'a') as f:
        f.write(f'{img_name},./train/{img_name}.jpg,{row.A},{row.B},{row.C},{row.D},{row.E},{row.F},{row.G},{row.H},{row.I},{row.J}\n')

9815it [08:13, 19.90it/s]
